# *Archivo para probar los prompt*

## Instalaciones

In [ ]:
%pip install pandas

## Obtener un paciente con su evolucion
Se hace una consulta a la BBDD para conseguir el historial de consultas de un paciente dado por su id de Paciente

Pacientes top 5 con mas consultas:
- id 172, Leo Martínez, consultas 3
- id 247, Luciana Gonzalez, consultas 3
- id 280, Matías Alejandro López Torres, consultas 3
- id 131, Lucas González, consultas 2
- id 153, Leonardo Martinez, consultas 2

In [ ]:
import sqlite3
import pandas as pd
import pprint

def query_to_database(query, db):
    # 1. Connect to the SQLite database
    conn = sqlite3.connect(db)

    try:
        df = pd.read_sql_query(query, conn)
        # print(df)
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()
        return df

# Obtener evolucion de un paciente
id_paciente = 172
sql_query_evolucion = f"""
SELECT * FROM evolucion WHERE id_paciente = {id_paciente};
"""
evolucion_df = query_to_database(sql_query_evolucion, '../01_setup/data/consultorio.db')

# Obtener datos personales del paciente
sql_query_paciente = f"""
SELECT * FROM paciente WHERE id_paciente = {id_paciente};
"""
paciente_df =query_to_database(sql_query_paciente, '../01_setup/data/consultorio.db')

# print("Datos paciente:")
# pprint.pprint(paciente_df) 
# print("Evolucion del paciente:")
# pprint.pprint(evolucion_df)


## Test de propt

### preparacion de datos de paciente y evoluciones

Formateo de clases

In [15]:
# configurar el parser de salida
from typing import Optional
from pydantic import BaseModel, Field

class Paciente(BaseModel):
    id_paciente: int = Field(...)
    apellido: str = Field(...)
    nombre: str = Field(...)
    fecha_nac: str = Field(...)
    sexo: str = Field(...)
    edad: str = Field(...)
    dni: str = Field(...)
    localidad: str = Field(...)
    obra_social: str = Field(...)
    afiliado_nro: str = Field(...)
    telefono: str = Field(...)
    telefono_numero: str = Field(...)
    email: str = Field(...)
    especialidad: str = Field(...)
    diagnostico: str = Field(...)
    enfermedad_base: str = Field(...)
    ant_perinatales: str = Field(...)
    ant_familiares: str = Field(...)
    registro: str = Field(...)
    fecha_registro: str = Field(...)
    
class Evolucion(BaseModel):
    id: int = Field(...)
    id_paciente: int = Field(...)
    fecha: str = Field(...)
    edad: int = Field(...)
    uni_edad: Optional[str] = Field(None)
    edad_anios: float = Field(...)
    edad_texto: str = Field(...)
    peso: float = Field(...)
    talla: float = Field(...)
    imc: float = Field(...)
    pc: float = Field(...)
    motivo: str = Field(...)
    conducta: Optional[str] = Field(None)

Creando objetos con los datos de paciente y evoluciones

In [ ]:
paciente = Paciente(**paciente_df.to_dict(orient='records')[0])
evoluciones = [Evolucion(**item) for item in evolucion_df.to_dict(orient='records')]

# pprint.pprint(paciente)
# pprint.pprint(evoluciones)

Paciente(id_paciente=172, apellido='Martínez', nombre='Leo', fecha_nac='2025-10-26', sexo='Varón', edad='6 meses', dni='70.493.370', localidad='Olavarría', obra_social='IOSFA', afiliado_nro='70494307', telefono='1145678901', telefono_numero='1145678901', email='leo.martinez@mail.com', especialidad='Pediátrico', diagnostico='Otra', enfermedad_base='Control de salud ', ant_perinatales='RNT 37 sem, parto, no internación en neonatología', ant_familiares='Antecedente materno de anemia en embarazo', registro='Secretaria', fecha_registro='Mon May 05 2025 15:23:22')


### Preparacion de Prompt

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    input=["paciente", "evoluciones"],
    template="""
    ### preparacion de datos de paciente y evoluciones
        Paciente: {paciente}
        Evoluciones: {evoluciones}
    ### Preparacion de Prompt
        Eres un asistente que ayuda a redactar notas medicas a partir de los datos del paciente y sus evoluciones. Tu tarea es generar un resumen de la historia clinica del paciente, incluyendo los datos relevantes de las evoluciones.
        Debes tener en cuenta:
            - que las evoluciones pueden tener diferentes formatos y contenidos, por lo que debes ser capaz de interpretar y sintetizar la informacion de manera coherente.
            - cruzar informacion de las diferentes evoluciones para identificar patrones, cambios y tendencias en la salud del paciente.
            - cruzar los antecendentes personales y familiares del paciente, asi como su diagnostico y enfermedad base.
    ### Salida esperada:
        Se espera que la salida sea un texto en uno o dos parrafos
        El texto debe ser claro y conciso, utilizando lenguaje medico apropiado.
        Al comenzar la redaccion, incluye una breve introduccion con los datos personales del paciente (nombre, edad, diagnostico -si es que lo tiene-, enfermedad base -si es que lo tiene-).
        Cada patron, tendencia o cambio identificado debe ser explicado de manera clara y breve precisa en un parrafo aparte, utilizando como dato de referencia la fecha de la evolucion en la que se toma la informacion.
    """,
    output_parser=StrOutputParser()
)

### Consulta al Modelo
preparando modelo

In [20]:
from langchain_openai import ChatOpenAI
import os

# Configurar la API key de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")
llm_model = "gpt-4o-mini"
llm = ChatOpenAI(openai_api_key=openai_api_key, model = llm_model, temperature=0.7)

obteniendo resultados

In [23]:
chain = prompt | llm | StrOutputParser()
input_data = {
    "paciente": paciente,
    "evoluciones": evoluciones
}
respuesta = chain.invoke(input = input_data)
pprint.pprint(respuesta)

('**Historia Clínica del Paciente**\n'
 '\n'
 '**Datos del Paciente:**\n'
 '- **Nombre:** Leo Martínez\n'
 '- **Edad:** 6 meses\n'
 '- **Diagnóstico:** Otra\n'
 '- **Enfermedad Base:** Control de salud\n'
 '- **DNI:** 70.493.370\n'
 '- **Localidad:** Olavarría\n'
 '- **Obra Social:** IOSFA\n'
 '- **Teléfono:** 1145678901\n'
 '- **Email:** leo.martinez@mail.com\n'
 '\n'
 '**Antecedentes:**\n'
 '- **Personales:** RNT 37 sem, parto, sin internación en neonatología.\n'
 '- **Familiares:** Antecedente materno de anemia durante el embarazo.\n'
 '\n'
 '**Evoluciones:**\n'
 '\n'
 '1. **Evolución del 09/05/2025:**\n'
 '   - **Edad:** 6 meses\n'
 '   - **Peso:** 7.8 kg\n'
 '   - **Talla:** 65.0 cm\n'
 '   - **IMC:** 18.46\n'
 '   - **Perímetro Cefálico (PC):** 43.0 cm\n'
 '   - **Motivo de Consulta:** Control de salud\n'
 '   - **Conducta:** Se recomienda continuar con la suplementación de sulfato '
 'ferroso (0.6 ml/día) y vitaminas ADC (0.3 ml/día). Se programó una ecografía '
 'de cadera a lo

In [ ]:
import pandas as pd

# Convertir la columna 'fecha' en la tabla de evolucion a formato de fecha
df_combinado['fecha'] = pd.to_datetime(df_combinado['fecha'], errors='coerce')

# Diccionario para almacenar los prompts de cada paciente
prompts_por_paciente = {}

# Agrupar el DataFrame por id_paciente
pacientes_agrupados = df_combinado.groupby('id_paciente')

# Iterar sobre cada grupo de paciente
for id_paciente, grupo_paciente in pacientes_agrupados:
    # Obtener los datos del paciente
    paciente = grupo_paciente.iloc[0]

    # Ordenar las evoluciones del paciente por fecha y eliminar los valores NaT
    # La eliminación de NaT previene el error 'AttributeError: float'
    evoluciones_ordenadas = grupo_paciente.dropna(subset=['fecha']).sort_values(by='fecha')

    # Construir la sección de evoluciones como una cadena de texto
    evoluciones_texto = ""
    if not evoluciones_ordenadas.empty:
        for index, evolucion in evoluciones_ordenadas.iterrows():
            evoluciones_texto += (
                f"- Fecha: {evolucion['fecha'].strftime('%Y-%m-%d')}\n"
                f"  Motivo: {evolucion['motivo']}\n"
                f"  Conducta: {evolucion['conducta']}\n\n"
            )
    else:
        evoluciones_texto = "No se encontraron evoluciones para este paciente.\n\n"

    # Crear el prompt completo del paciente
    prompt_completo = f"""Historia Clínica del Paciente:
- Nombre: {paciente['nombre']} {paciente['apellido']}
- DNI: {paciente['dni']}
- Fecha de Nacimiento: {paciente['fecha_nac']}
- Sexo: {paciente['sexo']}
- Especialidad: {paciente['especialidad']}
- Diagnóstico Principal: {paciente['diagnostico']}
- Enfermedad de Base: {paciente['enfermedad_base']}
- Antecedentes Perinatales: {paciente['ant_perinatales']}
- Antecedentes Familiares: {paciente['ant_familiares']}

Evoluciones del Paciente:
{evoluciones_texto}
"""
prompts_por_paciente[id_paciente] = prompt_completo

# Ejemplo de cómo se vería un prompt
print("--- Ejemplo de un prompt generado ---")
# Puedes cambiar el ID para ver el prompt de otro paciente
ejemplo_prompt = prompts_por_paciente.get(150, "No se encontró el paciente con el ID 2.")
print(ejemplo_prompt)


--- Ejemplo de un prompt generado ---
No se encontró el paciente con el ID 2.


In [ ]:
# Instalar openai para generar agente
%pip install openai

In [5]:
import openai
from dotenv import load_dotenv
import os

#Cargar las variables del archivo env con la key
#El archivo esta en la carpeta 02_prompt y posee el arhivo setup_openia.env
load_dotenv()

#Configurar la api key de openia
openai.api_key = os.getenv("OPENAI_key")